Analisis Sentimen Twitter terhadap Timnas Indonesia di Sea Games 2023 
Menggunakan Algoritma Naïve Bayes, Logistic Regression dan Random Forest.

Anggota Team :
1. Catur Agus Baktiar | jobdesc : Text PreProcessing | Universitas Muhammadiyah jakarta
2. Siti Saropah | jobdesc : Modelling | STMIK IKMI Cirebon
3. Siti Fatimah Nurjanah | jobdesc : Feature Engineering | Universitas Swadaya Gunung Jati
4. Muhammad Ganarya Nirwana | jobdesc: Data Scrapping | Universitas Muhammadiyah jakarta

# DATA ACQUISTION

## Crawling Data Twitter

In [ ]:
!pip install --upgrade git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-bakwfx88
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-bakwfx88
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 786815dd05681e2421cd03aa9acf5ab5c773bce9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install snscrape
import snscrape.modules.twitter as sntwitter



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('timnas since:2023-05-11 until:2023-05-14').get_items()):
    if i>10000:
        break
    attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])
    
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"])

In [ ]:
tweets_df

,User,Date Created,Number of Likes,Source of Tweet,Tweet
0,86Vahmie,2023-05-13 23:59:44+00:00,0,None,@ganjarpranowo wkwkwk pencitraan terus njar ga...
1,RosyaTV,2023-05-13 23:59:12+00:00,2,None,Nazar Indra Sjafri Jika Timnas Indonesia Juara...
2,AuisGrand,2023-05-13 23:53:14+00:00,0,None,Simalakama Timnas Basket di Semifinal \nhttps:...
3,OptaMola,2023-05-13 23:54:04+00:00,1,None,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,None,@PSSI Jadi jngan karna nama besar STY timnas k...
...,...,...,...,...,...
9585,tvOneNews,2023-05-11 00:02:07+00:00,3,None,3 Pemain Persib Bandung yang Menolak Persija J...
9586,mrzapf_,2023-05-11 00:01:42+00:00,0,None,@pandji Kenapa coldplay gak konser di JIS aja ...
9587,OghieGigs,2023-05-11 00:01:19+00:00,3,None,Well. Perancis ga turun dengan kekuatan terbai...
9588,tribunSUPERBALL,2023-05-11 00:00:00+00:00,0,None,Kementerian Pemuda dan Olahraga Republik Indon...


In [ ]:
tweets_df.to_csv('data_timnas_1.csv', index=False)

# DATA EXPLORATION

## PREPOCESSING DATA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
# Untuk mempermudah, simpan setiap objek agar dapat digunakan untuk pemodelan maupun deployment. Gunakan library Pickle
import pickle

%matplotlib inline

In [ ]:
# Install library Sastrawi
!pip -q install sastrawi

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
tweets_df = pd.read_csv('data_timnas_1.csv')
print(tweets_df)

                 User               Date Created  Number of Likes  \
0            86Vahmie  2023-05-13 23:59:44+00:00                0   
1             RosyaTV  2023-05-13 23:59:12+00:00                2   
2           AuisGrand  2023-05-13 23:53:14+00:00                0   
3            OptaMola  2023-05-13 23:54:04+00:00                1   
4        KurangiBodoh  2023-05-13 23:53:14+00:00                0   
...               ...                        ...              ...   
9585        tvOneNews  2023-05-11 00:02:07+00:00                3   
9586          mrzapf_  2023-05-11 00:01:42+00:00                0   
9587        OghieGigs  2023-05-11 00:01:19+00:00                3   
9588  tribunSUPERBALL  2023-05-11 00:00:00+00:00                0   
9589     BolaSportcom  2023-05-11 00:00:00+00:00                0   

      Source of Tweet                                              Tweet  
0                 NaN  @ganjarpranowo wkwkwk pencitraan terus njar ga...  
1                 NaN

In [ ]:
tweet = tweets_df['Tweet']

### CASE FOLDING

In [ ]:
def clean_text(tweet):
  #menghapus username twiter
  text=re.sub('@[^\s]+',' ', tweet)
  #menghapus https dan http 
  text=re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
  #menghilangkan tanda baca
  text = text.translate(str.maketrans(' ', ' ',string.punctuation))
  #mengganti karakter html dengan tanda petik
  text=re.sub('[^a-zA-Z]',' ',text)
  #mengganti line baru dengan spasi 
  text = re.sub("\n"," ",text)
  #mengubah ke huruf kecil
  text = text.lower()
  #menghapus single char
  text = re.sub(r"\b[a-zA-Z]\b"," ",text)
  #memisahkan dan menggabungkan kata
  text=' '.join(text.split())
  return text

In [ ]:
tweets_df['clean'] = [clean_text(i) for i in tweet]
tweet = tweets_df['clean']

In [ ]:
tweets_df.head(10)

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...
5,Salvatjoire,2023-05-13 23:53:09+00:00,1,NaN,@garagarabola_ semua cinta timnas,semua cinta timnas
6,hariankompas,2023-05-13 23:51:11+00:00,1,NaN,Timnas Indonesia harus segera memperbaiki perf...,timnas indonesia harus segera memperbaiki perf...
7,gamapila,2023-05-13 23:49:35+00:00,0,NaN,@MafiaWasit The next level iwan bule. Kalo dul...,the next level iwan bule kalo dulu ibul buat v...
8,Komar37936404,2023-05-13 23:49:27+00:00,1,NaN,@AjieNdoro @PSSI Ini pasti pembenci timnas...wkwk,ini pasti pembenci timnaswkwk
9,BryanMyBoy2,2023-05-13 23:48:50+00:00,0,NaN,Masi tidak percaya dg terjadinya gol ke 3 Timn...,masi tidak percaya dg terjadinya gol ke timnas...


In [ ]:
print(tweets_df)

                 User               Date Created  Number of Likes  \
0            86Vahmie  2023-05-13 23:59:44+00:00                0   
1             RosyaTV  2023-05-13 23:59:12+00:00                2   
2           AuisGrand  2023-05-13 23:53:14+00:00                0   
3            OptaMola  2023-05-13 23:54:04+00:00                1   
4        KurangiBodoh  2023-05-13 23:53:14+00:00                0   
...               ...                        ...              ...   
9585        tvOneNews  2023-05-11 00:02:07+00:00                3   
9586          mrzapf_  2023-05-11 00:01:42+00:00                0   
9587        OghieGigs  2023-05-11 00:01:19+00:00                3   
9588  tribunSUPERBALL  2023-05-11 00:00:00+00:00                0   
9589     BolaSportcom  2023-05-11 00:00:00+00:00                0   

      Source of Tweet                                              Tweet  \
0                 NaN  @ganjarpranowo wkwkwk pencitraan terus njar ga...   
1                 N

### Tokenezing

In [ ]:
from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer(r'\w+|$[0-9]+|S+')
tweets_df['Token'] = tweets_df['clean'].apply(regexp.tokenize)

In [ ]:
tweets_df.head(10)

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans..."
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia..."
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]"
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,..."
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,..."
5,Salvatjoire,2023-05-13 23:53:09+00:00,1,NaN,@garagarabola_ semua cinta timnas,semua cinta timnas,"[semua, cinta, timnas]"
6,hariankompas,2023-05-13 23:51:11+00:00,1,NaN,Timnas Indonesia harus segera memperbaiki perf...,timnas indonesia harus segera memperbaiki perf...,"[timnas, indonesia, harus, segera, memperbaiki..."
7,gamapila,2023-05-13 23:49:35+00:00,0,NaN,@MafiaWasit The next level iwan bule. Kalo dul...,the next level iwan bule kalo dulu ibul buat v...,"[the, next, level, iwan, bule, kalo, dulu, ibu..."
8,Komar37936404,2023-05-13 23:49:27+00:00,1,NaN,@AjieNdoro @PSSI Ini pasti pembenci timnas...wkwk,ini pasti pembenci timnaswkwk,"[ini, pasti, pembenci, timnaswkwk]"
9,BryanMyBoy2,2023-05-13 23:48:50+00:00,0,NaN,Masi tidak percaya dg terjadinya gol ke 3 Timn...,masi tidak percaya dg terjadinya gol ke timnas...,"[masi, tidak, percaya, dg, terjadinya, gol, ke..."


### Slang Word Normalization

In [ ]:
normalized_word = pd.read_csv("normalisasi.csv", encoding='latin1')

normalized_word_dict={}
for index, row in normalized_word.iterrows():
  if row[0] not in normalized_word_dict:
    normalized_word_dict[row[0]] = row[1]
    
def normalized_term(document):
  return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

tweets_df['Normalisasi'] = tweets_df['Token'].apply(normalized_term)
tweets_df

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[wkwkwk, pencitraan, terus, njar, ganjar, fans..."
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, jika, timnas, indonesia..."
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, di, semifinal]"
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,..."
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,...","[jadi, jngan, karena, nama, besar, sty, timnas..."
...,...,...,...,...,...,...,...,...
9585,tvOneNews,2023-05-11 00:02:07+00:00,3,NaN,3 Pemain Persib Bandung yang Menolak Persija J...,pemain persib bandung yang menolak persija jak...,"[pemain, persib, bandung, yang, menolak, persi...","[pemain, persib, bandung, yang, menolak, persi..."
9586,mrzapf_,2023-05-11 00:01:42+00:00,0,NaN,@pandji Kenapa coldplay gak konser di JIS aja ...,kenapa coldplay gak konser di jis aja sih timn...,"[kenapa, coldplay, gak, konser, di, jis, aja, ...","[kenapa, coldplay, tidak, konser, di, jis, saj..."
9587,OghieGigs,2023-05-11 00:01:19+00:00,3,NaN,Well. Perancis ga turun dengan kekuatan terbai...,well perancis ga turun dengan kekuatan terbaik...,"[well, perancis, ga, turun, dengan, kekuatan, ...","[well, perancis, tidak, turun, dengan, kekuata..."
9588,tribunSUPERBALL,2023-05-11 00:00:00+00:00,0,NaN,Kementerian Pemuda dan Olahraga Republik Indon...,kementerian pemuda dan olahraga republik indon...,"[kementerian, pemuda, dan, olahraga, republik,...","[kementerian, pemuda, dan, olahraga, republik,..."


### Filtering (Stopword Removal)

In [ ]:
from nltk.corpus import stopwords
stopword = stopwords.words('indonesian')
txt_stopword = pd.read_csv("stopwords.txt", names=["stopwords"], header = None)

stopword.extend(['wkwkwk', 'ke', 'amp','diginii'])

stopword.extend(txt_stopword["stopwords"][0].split(' '))
stopword = set(stopword)

def stopwords(tweet):
  tweet = [word for word in tweet if word not in stopword]
  return tweet
tweets_df['Stopwords'] = tweets_df['Normalisasi'].apply(stopwords)
tweets_df

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi,Stopwords
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[pencitraan, njar, ganjar, fans, timnas, lupa,..."
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, timnas, indonesia, juar..."
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, semifinal]"
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,..."
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,...","[jadi, jngan, karena, nama, besar, sty, timnas...","[jngan, nama, sty, timnas, taruhan, nya, cm, s..."
...,...,...,...,...,...,...,...,...,...
9585,tvOneNews,2023-05-11 00:02:07+00:00,3,NaN,3 Pemain Persib Bandung yang Menolak Persija J...,pemain persib bandung yang menolak persija jak...,"[pemain, persib, bandung, yang, menolak, persi...","[pemain, persib, bandung, yang, menolak, persi...","[pemain, persib, bandung, menolak, persija, ja..."
9586,mrzapf_,2023-05-11 00:01:42+00:00,0,NaN,@pandji Kenapa coldplay gak konser di JIS aja ...,kenapa coldplay gak konser di jis aja sih timn...,"[kenapa, coldplay, gak, konser, di, jis, aja, ...","[kenapa, coldplay, tidak, konser, di, jis, saj...","[coldplay, konser, jis, sih, timnas, main, sol..."
9587,OghieGigs,2023-05-11 00:01:19+00:00,3,NaN,Well. Perancis ga turun dengan kekuatan terbai...,well perancis ga turun dengan kekuatan terbaik...,"[well, perancis, ga, turun, dengan, kekuatan, ...","[well, perancis, tidak, turun, dengan, kekuata...","[well, perancis, turun, kekuatan, terbaiknya, ..."
9588,tribunSUPERBALL,2023-05-11 00:00:00+00:00,0,NaN,Kementerian Pemuda dan Olahraga Republik Indon...,kementerian pemuda dan olahraga republik indon...,"[kementerian, pemuda, dan, olahraga, republik,...","[kementerian, pemuda, dan, olahraga, republik,...","[kementerian, pemuda, olahraga, republik, indo..."


### Stemming

In [ ]:
!pip install swifter
print("completed")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
completed


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory=StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

tweet = tweets_df['Stopwords']
for doc in tweet:
  for term in doc:
    if term not in term_dict:
      term_dict[term]=' '
  print(len(term_dict))
  print("-----------------------------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  print(term,":",term_dict[term])

  print(len(term_dict))
  print("-----------------------------")

Output streaming akan dipotong hingga 5000 baris terakhir.
14074
-----------------------------
saito : saito
14074
-----------------------------
beraaat : beraaat
14074
-----------------------------
timnasinikah : timnasinikah
14074
-----------------------------
nmanya : nmanya
14074
-----------------------------
lebarrrr : lebarrrr
14074
-----------------------------
acaranya : acara
14074
-----------------------------
prik : prik
14074
-----------------------------
ladiesnya : ladiesnya
14074
-----------------------------
nicknamenya : nicknamenya
14074
-----------------------------
matanya : mata
14074
-----------------------------
sluruh : sluruh
14074
-----------------------------
syulitpesimis : syulitpesimis
14074
-----------------------------
tros : tros
14074
-----------------------------
trakhir : trakhir
14074
-----------------------------
special : special
14074
-----------------------------
brighton : brighton
14074
-----------------------------
bismilah : bismilah
14074
-

In [ ]:
def get_stemmed_term(doc):
  return [term_dict[term] for term in doc]

tweets_df['Stemmer'] = tweets_df['Stopwords'].apply(get_stemmed_term)
tweets_df

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi,Stopwords,Stemmer
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[pencitraan, njar, ganjar, fans, timnas, lupa,...","[citra, njar, ganjar, fans, timnas, lupa, dosa..."
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, timnas, indonesia, juar...","[nazar, indra, sjafri, timnas, indonesia, juar..."
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, semifinal]","[simalakama, timnas, basket, semifinal]"
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,..."
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,...","[jadi, jngan, karena, nama, besar, sty, timnas...","[jngan, nama, sty, timnas, taruhan, nya, cm, s...","[jngan, nama, sty, timnas, taruh, nya, cm, sbu..."
...,...,...,...,...,...,...,...,...,...,...
9585,tvOneNews,2023-05-11 00:02:07+00:00,3,NaN,3 Pemain Persib Bandung yang Menolak Persija J...,pemain persib bandung yang menolak persija jak...,"[pemain, persib, bandung, yang, menolak, persi...","[pemain, persib, bandung, yang, menolak, persi...","[pemain, persib, bandung, menolak, persija, ja...","[main, persib, bandung, tolak, persija, jakart..."
9586,mrzapf_,2023-05-11 00:01:42+00:00,0,NaN,@pandji Kenapa coldplay gak konser di JIS aja ...,kenapa coldplay gak konser di jis aja sih timn...,"[kenapa, coldplay, gak, konser, di, jis, aja, ...","[kenapa, coldplay, tidak, konser, di, jis, saj...","[coldplay, konser, jis, sih, timnas, main, sol...","[coldplay, konser, jis, sih, timnas, main, sol..."
9587,OghieGigs,2023-05-11 00:01:19+00:00,3,NaN,Well. Perancis ga turun dengan kekuatan terbai...,well perancis ga turun dengan kekuatan terbaik...,"[well, perancis, ga, turun, dengan, kekuatan, ...","[well, perancis, tidak, turun, dengan, kekuata...","[well, perancis, turun, kekuatan, terbaiknya, ...","[well, perancis, turun, kuat, baik, lepas, klu..."
9588,tribunSUPERBALL,2023-05-11 00:00:00+00:00,0,NaN,Kementerian Pemuda dan Olahraga Republik Indon...,kementerian pemuda dan olahraga republik indon...,"[kementerian, pemuda, dan, olahraga, republik,...","[kementerian, pemuda, dan, olahraga, republik,...","[kementerian, pemuda, olahraga, republik, indo...","[menteri, pemuda, olahraga, republik, indonesi..."


##Labelling

In [ ]:
#Lexicon Based
!pip install VaderSentiment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
scores = [analyser.polarity_scores(x) for x in tweets_df['Tweet']]
print(scores)
tweets_df['compound_Score'] = [x['compound'] for x in scores]


[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.936, 'pos': 0.064, 'compound': 0.4767}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'

In [ ]:
tweets_df.head()

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi,Stopwords,Stemmer,compound_Score
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[pencitraan, njar, ganjar, fans, timnas, lupa,...","[citra, njar, ganjar, fans, timnas, lupa, dosa...",0.0
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, timnas, indonesia, juar...","[nazar, indra, sjafri, timnas, indonesia, juar...",0.0
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, semifinal]","[simalakama, timnas, basket, semifinal]",0.0
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...",0.0
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,...","[jadi, jngan, karena, nama, besar, sty, timnas...","[jngan, nama, sty, timnas, taruhan, nya, cm, s...","[jngan, nama, sty, timnas, taruh, nya, cm, sbu...",0.0


In [ ]:
tweets_df.nsmallest(10, ['compound_Score'])

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi,Stopwords,Stemmer,compound_Score
2820,Givms25,2023-05-13 10:59:25+00:00,2,NaN,GOLLLLLLLLLLLLLL GW JADI MOOD LAGI NONTON TIMN...,gollllllllllllll gw jadi mood lagi nonton timn...,"[gollllllllllllll, gw, jadi, mood, lagi, nonto...","[gollllllllllllll, gw, jadi, mood, lagi, nonto...","[gollllllllllllll, gw, mood, nonton, timnas, c...","[gollllllllllllll, gw, mood, nonton, timnas, c...",-0.9948
8458,ululazmisyfr,2023-05-11 11:54:01+00:00,1,NaN,definisi doa jadi kenyataan wkwkwkwk jadi kein...,definisi doa jadi kenyataan wkwkwkwk jadi kein...,"[definisi, doa, jadi, kenyataan, wkwkwkwk, jad...","[definisi, doa, jadi, kenyataan, wkwkwkwk, jad...","[definisi, doa, kenyataan, wkwkwkwk, keinget, ...","[definisi, doa, nyata, wkwkwkwk, keinget, thre...",-0.9750
7804,wargakalisari,2023-05-11 13:39:40+00:00,0,NaN,Ketemu jepang😭😭😭😭 sekelas van dijk aja dibuat ...,ketemu jepang sekelas van dijk aja dibuat nung...,"[ketemu, jepang, sekelas, van, dijk, aja, dibu...","[ketemu, jepang, sekelas, van, dijk, saja, dib...","[ketemu, jepang, sekelas, van, dijk, nunggung,...","[ketemu, jepang, kelas, van, dijk, nunggung, s...",-0.9744
4828,sincossetannn,2023-05-12 16:12:44+00:00,0,NaN,nt timnas emel😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥😥,nt timnas emel,"[nt, timnas, emel]","[nt, timnas, emel]","[nt, timnas, emel]","[nt, timnas, emel]",-0.9688
1921,funf_lala,2023-05-13 11:28:40+00:00,0,NaN,"Aaaaaaaaaaaa ak ngereog d sini aja deh huhuhu,...",aaaaaaaaaaaa ak ngereog sini aja deh huhuhu ha...,"[aaaaaaaaaaaa, ak, ngereog, sini, aja, deh, hu...","[aaaaaaaaaaaa, saya, ngereog, sini, saja, deh,...","[aaaaaaaaaaaa, ngereog, deh, huhuhu, putus, ha...","[aaaaaaaaaaaa, ngereog, deh, huhuhu, putus, ha...",-0.9559
7030,haikudrifter,2023-05-12 01:51:46+00:00,0,NaN,"@grealinho @idextratime Hah, Pep main untuk Ti...",hah pep main untuk timnas indonesia,"[hah, pep, main, untuk, timnas, indonesia]","[hah, pep, main, untuk, timnas, indonesia]","[hah, pep, main, timnas, indonesia]","[hah, pep, main, timnas, indonesia]",-0.9493
2596,narsdiary,2023-05-13 11:04:21+00:00,0,NaN,EMANG PASRAH ADALAH KUNCI KEMENANGAN TIMNAS IN...,emang pasrah adalah kunci kemenangan timnas in...,"[emang, pasrah, adalah, kunci, kemenangan, tim...","[emang, pasrah, adalah, kunci, kemenangan, tim...","[emang, pasrah, kunci, kemenangan, timnas, ind...","[emang, pasrah, kunci, menang, timnas, indonesia]",-0.9382
2767,kimjoonra__,2023-05-13 11:01:44+00:00,0,NaN,ALHAMDULILAH YA ALLAH TIMNAS INDONESIA😭😭😭😭😭,alhamdulilah ya allah timnas indonesia,"[alhamdulilah, ya, allah, timnas, indonesia]","[alhamdulilah, ya, allah, timnas, indonesia]","[alhamdulilah, ya, allah, timnas, indonesia]","[alhamdulilah, ya, allah, timnas, indonesia]",-0.9382
8107,wijimoharwan,2023-05-11 12:31:37+00:00,1,NaN,"@PSSI Tenangke pikirmu min, sing sabar ya timn...",tenangke pikirmu min sing sabar ya timnas indo...,"[tenangke, pikirmu, min, sing, sabar, ya, timn...","[tenangke, pikirmu, admin, sing, sabar, ya, ti...","[tenangke, pikirmu, admin, sing, sabar, ya, ti...","[tenangke, pikir, admin, sing, sabar, ya, timn...",-0.9382
4691,deinschatzi_,2023-05-12 18:25:43+00:00,0,NaN,🗣️ bisa ga stop jadi kpopers? \n❌❌\n\n🗣️ yauda...,bisa ga stop jadi kpopers yauda stop suka sama...,"[bisa, ga, stop, jadi, kpopers, yauda, stop, s...","[bisa, tidak, stop, jadi, kpopers, yauda, stop...","[stop, kpopers, yauda, stop, suka, fc, bayern,...","[stop, kpopers, yauda, stop, suka, fc, bayern,...",-0.9287


In [ ]:
#Compound Score Lexicon based
tweets_df.loc[tweets_df['compound_Score'] < 0, 'Sentiments'] = 'Negatif'

tweets_df.loc[tweets_df['compound_Score'] == 0, 'Sentiments'] = 'Netral'

tweets_df.loc[tweets_df['compound_Score'] > 0, 'Sentiments'] = 'Positif'
tweets_df.head()

,User,Date Created,Number of Likes,Source of Tweet,Tweet,clean,Token,Normalisasi,Stopwords,Stemmer,compound_Score,Sentiments
0,86Vahmie,2023-05-13 23:59:44+00:00,0,NaN,@ganjarpranowo wkwkwk pencitraan terus njar ga...,wkwkwk pencitraan terus njar ganjar fans timna...,"[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[wkwkwk, pencitraan, terus, njar, ganjar, fans...","[pencitraan, njar, ganjar, fans, timnas, lupa,...","[citra, njar, ganjar, fans, timnas, lupa, dosa...",0.0,Netral
1,RosyaTV,2023-05-13 23:59:12+00:00,2,NaN,Nazar Indra Sjafri Jika Timnas Indonesia Juara...,nazar indra sjafri jika timnas indonesia juara...,"[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, jika, timnas, indonesia...","[nazar, indra, sjafri, timnas, indonesia, juar...","[nazar, indra, sjafri, timnas, indonesia, juar...",0.0,Netral
2,AuisGrand,2023-05-13 23:53:14+00:00,0,NaN,Simalakama Timnas Basket di Semifinal \nhttps:...,simalakama timnas basket di semifinal,"[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, di, semifinal]","[simalakama, timnas, basket, semifinal]","[simalakama, timnas, basket, semifinal]",0.0,Netral
3,OptaMola,2023-05-13 23:54:04+00:00,1,NaN,#SEAGames2023 \nSEPAK TAKRAW Men's Team Double...,seagames sepak takraw mens team doubles gold i...,"[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...","[seagames, sepak, takraw, mens, team, doubles,...",0.0,Netral
4,KurangiBodoh,2023-05-13 23:53:14+00:00,0,NaN,@PSSI Jadi jngan karna nama besar STY timnas k...,jadi jngan karna nama besar sty timnas kita jd...,"[jadi, jngan, karna, nama, besar, sty, timnas,...","[jadi, jngan, karena, nama, besar, sty, timnas...","[jngan, nama, sty, timnas, taruhan, nya, cm, s...","[jngan, nama, sty, timnas, taruh, nya, cm, sbu...",0.0,Netral


In [ ]:
tweets_df.to_csv('data_timnas2023.csv', index=False)

#Feature Engineering

In [ ]:
# Pisahkan kolom fitur dan target
X = tweets_df['clean']
y = tweets_df['Sentiments']

In [ ]:
X

0       wkwkwk pencitraan terus njar ganjar fans timna...
1       nazar indra sjafri jika timnas indonesia juara...
2                   simalakama timnas basket di semifinal
3       seagames sepak takraw mens team doubles gold i...
4       jadi jngan karna nama besar sty timnas kita jd...
                              ...                        
9585    pemain persib bandung yang menolak persija jak...
9586    kenapa coldplay gak konser di jis aja sih timn...
9587    well perancis ga turun dengan kekuatan terbaik...
9588    kementerian pemuda dan olahraga republik indon...
9589    kementerian pemuda dan olahraga republik indon...
Name: clean, Length: 9590, dtype: object

In [ ]:
y

0        Netral
1        Netral
2        Netral
3        Netral
4        Netral
         ...   
9585     Netral
9586    Positif
9587    Positif
9588     Netral
9589     Netral
Name: Sentiments, Length: 9590, dtype: object

##Feature Extraction (Bag of Words & N-Gram)

In [ ]:
'''
Convert a collection of text documents to a matrix of token counts.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
'''
from sklearn.feature_extraction.text import CountVectorizer

# BoW - Unigram
bow = CountVectorizer(ngram_range=(1,1))
bow.fit(X)

CountVectorizer()

In [ ]:
# Melihat jumlah fitur
print(len(bow.get_feature_names_out()))

14744


In [ ]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(bow.get_feature_names_out())

['aa' 'aaa' 'aaaa' ... 'zuudi' 'zz' 'zzzzz']


In [ ]:
# Melihat matriks jumlah fitur
# Data ini siap untuk dimasukkan dalam proses pemodelan (machine learning)

X_bow = bow.transform(X).toarray()
X_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
data_bow = pd.DataFrame(X_bow, columns=bow.get_feature_names_out())
data_bow

,aa,aaa,aaaa,aaaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaak,aaaaaaaaaakkkk,aaaahhhhh,aaakk,aaamiinnn,...,zonauang,zonk,zubi,zubimendi,zulfiandi,zulham,zuu,zuudi,zz,zzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
with open('bow_feature.pickle', 'wb') as output:
  pickle.dump(X_bow, output)

##Feature Extraction (TF-IDF & N-Gram)

In [ ]:
'''
Convert a collection of raw documents to a matrix of TF-IDF features
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(X)

TfidfVectorizer()

In [ ]:
# Melihat Jumlah Fitur
print(len(tf_idf.get_feature_names_out()))

14744


In [ ]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(tf_idf.get_feature_names_out())

['aa' 'aaa' 'aaaa' ... 'zuudi' 'zz' 'zzzzz']


In [ ]:
# Melihat matriks jumlah token
# Data ini siap untuk dimasukkan dalam proses pemodelan (machine learning)

X_tf_idf = tf_idf.transform(X).toarray()
X_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Melihat matriks jumlah token menggunakan TF IDF, lihat perbedaannya dengan metode BoW
# Data ini siap untuk dimasukkan dalam proses pemodelan (machine learning)

data_tf_idf = pd.DataFrame(X_tf_idf, columns=tf_idf.get_feature_names_out())
data_tf_idf

,aa,aaa,aaaa,aaaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaak,aaaaaaaaaakkkk,aaaahhhhh,aaakk,aaamiinnn,...,zonauang,zonk,zubi,zubimendi,zulfiandi,zulham,zuu,zuudi,zz,zzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
with open('tf_idf_feature.pickle', 'wb') as output:
  pickle.dump(X_tf_idf, output)

##Feature Selection (Chi Square)

In [ ]:
# Mengubah nilai data tabular tf-idf menjadi array agar dapat dijalankan pada proses seleksi fitur
X = np.array(data_tf_idf)
y = np.array(y)

In [ ]:
'''
Select features according to the k highest scores.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

Compute chi-squared stats between each non-negative feature and class.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html
'''

from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 

# Ten features with highest chi-squared statistics are selected 
chi2_features = SelectKBest(chi2, k=100) 
X_kbest_features = chi2_features.fit_transform(X, y) 
  
# Reduced features 
print('Original feature number:', X.shape[1]) 
print('Reduced feature number:', X_kbest_features.shape[1])

Original feature number: 14744
Reduced feature number: 100


In [ ]:
# chi2_features.scores_ adalah nilai chi-square, semakin tinggi nilainya maka semakin baik fiturnya
data_chi2 = pd.DataFrame(chi2_features.scores_, columns=['nilai'])
data_chi2

,nilai
0,2.174082
1,0.663816
2,2.780482
3,6.113241
4,2.742865
...,...
14739,0.088560
14740,1.391620
14741,1.391620
14742,0.151424


In [ ]:
# Menampilkan fitur beserta nilainya
feature = tf_idf.get_feature_names_out()
data_chi2['fitur'] = feature
data_chi2

,nilai,fitur
0,2.174082,aa
1,0.663816,aaa
2,2.780482,aaaa
3,6.113241,aaaaaaaaa
4,2.742865,aaaaaaaaaaaa
...,...,...
14739,0.088560,zulham
14740,1.391620,zuu
14741,1.391620,zuudi
14742,0.151424,zz


In [ ]:
# Mengurutkan fitur terbaik
data_chi2.sort_values(by='nilai', ascending=False)

,nilai,fitur
4502,166.154204,gg
12579,154.699488,sore
2433,110.621194,congrats
5390,94.417454,injury
3522,76.520667,doa
...,...,...
9629,0.004606,nontonin
14030,0.004115,update
6851,0.003851,kontribusi
7651,0.003387,males


In [ ]:
# Menampilkan mask pada feature yang diseleksi
# False berarti fitur tidak terpilih dan True berarti fitur terpilih
mask = chi2_features.get_support()
mask

array([False, False, False, ..., False, False, False])

In [ ]:
# Menampilkan fitur-fitur terpilih berdasarkan mask atau nilai tertinggi yang sudah dikalkulasi pada Chi-Square
new_feature = []

for bool, f in zip(mask, feature):
  if bool:
    new_feature.append(f)
  selected_feature = new_feature

selected_feature

['abuse',
 'anti',
 'asia',
 'bad',
 'banget',
 'banned',
 'be',
 'best',
 'bonus',
 'bug',
 'bully',
 'congrats',
 'congratulation',
 'congratulations',
 'defense',
 'dicetak',
 'doa',
 'dreams',
 'ez',
 'fight',
 'fighting',
 'fire',
 'for',
 'games',
 'garudanusantara',
 'gg',
 'gokill',
 'gold',
 'good',
 'grand',
 'gratis',
 'grup',
 'haha',
 'hahaha',
 'hard',
 'hero',
 'hoax',
 'hopeless',
 'if',
 'indonesia',
 'injury',
 'job',
 'join',
 'killing',
 'klimaks',
 'ladies',
 'like',
 'lose',
 'luck',
 'manyala',
 'medal',
 'membalaskan',
 'mlbb',
 'nah',
 'nangisin',
 'ngebon',
 'nice',
 'no',
 'of',
 'piala',
 'please',
 'points',
 'prabowo',
 'pressure',
 'pride',
 'problem',
 'respect',
 'riot',
 'sad',
 'sea',
 'semifinal',
 'share',
 'shoot',
 'sob',
 'sok',
 'solid',
 'sore',
 'sorry',
 'still',
 'stop',
 'stress',
 'stronger',
 'subcribe',
 'support',
 'supporter',
 'thank',
 'thanks',
 'time',
 'top',
 'try',
 'uncle',
 'underestimate',
 'united',
 'vietnam',
 'war',
 'wel

In [ ]:
# Cara melihat vocab yang dihasilkan oleh TF_IDF
# tf_idf.vocabulary_ 

kbest_feature = {} # Buat dictionary kosong

for (k,v) in tf_idf.vocabulary_.items():    # Iterasi untuk mengulangi vocab yang dihasilkan TF_IDF
  if k in selected_feature:                 # Cek apakah fitur termasuk k fitur yang diseleksi
    kbest_feature[k] = v                    # Jika iya, simpan fitur tersebut pada dictionary kosong diatas

In [ ]:
kbest_feature

{'indonesia': 5281,
 'sea': 11806,
 'games': 4302,
 'semifinal': 12069,
 'gold': 4613,
 'grup': 4706,
 'vietnam': 14159,
 'medal': 7912,
 'piala': 10680,
 'congrats': 2433,
 'prabowo': 10914,
 'anti': 540,
 'pride': 10972,
 'asia': 676,
 'doa': 3522,
 'support': 12839,
 'sore': 12579,
 'banget': 999,
 'dicetak': 2917,
 'injury': 5390,
 'time': 13535,
 'haha': 4807,
 'united': 14010,
 'win': 14418,
 'sok': 12552,
 'best': 1583,
 'join': 5764,
 'respect': 11387,
 'still': 12706,
 'top': 13703,
 'supporter': 12841,
 'congratulations': 2441,
 'mlbb': 8794,
 'please': 10784,
 'of': 9798,
 'good': 4639,
 'stop': 12712,
 'gg': 4502,
 'fight': 4060,
 'luck': 7488,
 'nah': 9043,
 'thank': 13448,
 'fighting': 4061,
 'you': 14678,
 'for': 4144,
 'be': 1151,
 'like': 7362,
 'bonus': 1853,
 'grand': 4680,
 'solid': 12556,
 'sorry': 12583,
 'hahaha': 4810,
 'stronger': 12735,
 'nice': 9546,
 'try': 13795,
 'problem': 10996,
 'share': 12292,
 'bully': 2000,
 'killing': 6626,
 'thanks': 13449,
 'fire'

In [ ]:
# Menampilkan fitur-fitur yang sudah diseleksi 
# Beserta nilai vektornya pada keseluruhan data untuk dijalankan pada proses machine learning

# Hanya k fitur yang terpilih sesuai parameter k yang ditentukan sebelumnya

data_selected_feature = pd.DataFrame(X_kbest_features, columns=selected_feature)
data_selected_feature

,abuse,anti,asia,bad,banget,banned,be,best,bonus,bug,...,uncle,underestimate,united,vietnam,war,well,win,wow,yes,you
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
9586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.405263,0.0,0.0,0.0
9587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.222219,0.000000,0.0,0.0,0.0
9588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
with open('kbest_feature.pickle', 'wb') as output:
  pickle.dump(kbest_feature, output)

#Modelling

##Naive Bayes

In [ ]:
'''
Supervised learning in Sklearn
https://scikit-learn.org/stable/supervised_learning.html
'''
from sklearn.naive_bayes import MultinomialNB           # Pilih salah satu algoritma supervised learning. Menggunakan algoritma Naive Bayes
from sklearn.model_selection import train_test_split    # Digunakan untuk memisahkan data uji dan data latih
from joblib import dump                                 # Digunakan untuk menyimpan model yang telah dilatih

In [ ]:
# Proses memisahkan data uji dan data latih. Perbandingan 80% untuk data latih, 20% untuk data uji
# Random_state digunakan untuk internal random generator
# Gunakan fitur (X) hasil seleksi fitur

# Split arrays or matrices into random train and test subsets.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.2, random_state=40)

In [ ]:
# Training the model
algorithm = MultinomialNB()               # Load algoritma pembelajaran
model = algorithm.fit(X_train, y_train)   # Fitkan (latih) algoritma pada data latih & label latih 

# Simpan model hasil traning
dump(model, filename='model_1.joblib')

['model_1.joblib']

In [ ]:
# Gunakan model yang telah di latih untuk memprediksi label pada data uji
model_pred = model.predict(X_test)

# Tampilkan hasil prediksi label dari model
model_pred

array(['Netral', 'Netral', 'Netral', ..., 'Netral', 'Netral', 'Netral'],
      dtype='<U7')

In [ ]:
# Tampilkan label sebenarnya pada data uji (actual label)
y_test

array(['Netral', 'Netral', 'Positif', ..., 'Negatif', 'Netral', 'Negatif'],
      dtype=object)

In [ ]:
# Hitung jumlah data yang berhasil di prediksi model & jumlah data yang salah di prediksi
prediksi_benar = (model_pred == y_test).sum()
prediksi_salah = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

accuracy = prediksi_benar / (prediksi_benar + prediksi_salah)*100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 1485
Jumlah prediksi salah	: 433
Akurasi pengujian	: 77.42440041710114 %


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, model_pred)
print('Confusion matrix:\n', cm)

Confusion matrix:
 [[   1  148    2]
 [   0 1450    0]
 [   0  283   34]]


In [ ]:
from sklearn.metrics import classification_report

print('Classification report:\n', classification_report(y_test, model_pred))

Classification report:
               precision    recall  f1-score   support

     Negatif       1.00      0.01      0.01       151
      Netral       0.77      1.00      0.87      1450
     Positif       0.94      0.11      0.19       317

    accuracy                           0.77      1918
   macro avg       0.91      0.37      0.36      1918
weighted avg       0.82      0.77      0.69      1918



In [ ]:
# Cross Validation

from sklearn.model_selection import ShuffleSplit    # bisa pilih beberapa teknik cross validation
from sklearn.model_selection import cross_val_score # untuk mengetahui performa model pada cross validation

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=50)

cv_accuracy = (cross_val_score(model, X_kbest_features, y, cv=cv, scoring='accuracy'))
avg_accuracy = np.mean(cv_accuracy)

print('Akurasi setiap split:', cv_accuracy, '\n')
print('Rata-rata akurasi pada cross validation:', avg_accuracy)

Akurasi setiap split: [0.76955162 0.75391032 0.77476538 0.76485923 0.76850886 0.77632951
 0.76120959 0.76694473 0.77059437 0.76277372] 

Rata-rata akurasi pada cross validation: 0.7669447340980188


In [ ]:
evaluation = pd.DataFrame({'Model': [],
                           'accuracy': []
                           })

In [ ]:
r = evaluation.shape[0]
evaluation.loc[r] = ['Naive Bayes', accuracy]
evaluation

,Model,accuracy
0,Naive Bayes,77.4244


##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression     # Pilih salah satu algoritma supervised learning. Menggunakan algoritma Logistic Regression
from sklearn.model_selection import train_test_split    # Digunakan untuk memisahkan data uji dan data latih
from joblib import dump                                 # Digunakan untuk menyimpan model yang telah dilatih

In [ ]:
# Proses memisahkan data uji dan data latih. Perbandingan 80% untuk data latih, 20% untuk data uji
# Random_state digunakan untuk internal random generator
# Gunakan fitur (X) hasil seleksi fitur

# Split arrays or matrices into random train and test subsets.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.2, random_state=40)

In [ ]:
# Training the model
algorithm = LogisticRegression()              # Load algoritma pembelajaran
model = algorithm.fit(X_train, y_train)   # Fitkan (latih) algoritma pada data latih & label latih 

# Simpan model hasil traning
dump(model, filename='model_2.joblib')

['model_2.joblib']

In [ ]:
# Gunakan model yang telah di latih untuk memprediksi label pada data uji
model_pred = model.predict(X_test)

# Tampilkan hasil prediksi label dari model
model_pred

array(['Netral', 'Netral', 'Netral', ..., 'Netral', 'Netral', 'Netral'],
      dtype=object)

In [ ]:
# Tampilkan label sebenarnya pada data uji (actual label)
y_test

array(['Netral', 'Netral', 'Positif', ..., 'Negatif', 'Netral', 'Negatif'],
      dtype=object)

In [ ]:
# Hitung jumlah data yang berhasil di prediksi model & jumlah data yang salah di prediksi
prediksi_benar = (model_pred == y_test).sum()
prediksi_salah = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

accuracy = prediksi_benar / (prediksi_benar + prediksi_salah)*100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 1523
Jumlah prediksi salah	: 395
Akurasi pengujian	: 79.40563086548488 %


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, model_pred)
print('Confusion matrix:\n', cm)

Confusion matrix:
 [[   9  138    4]
 [   0 1450    0]
 [   2  251   64]]


In [ ]:
from sklearn.metrics import classification_report

print('Classification report:\n', classification_report(y_test, model_pred))

Classification report:
               precision    recall  f1-score   support

     Negatif       0.82      0.06      0.11       151
      Netral       0.79      1.00      0.88      1450
     Positif       0.94      0.20      0.33       317

    accuracy                           0.79      1918
   macro avg       0.85      0.42      0.44      1918
weighted avg       0.82      0.79      0.73      1918



In [ ]:
# Cross Validation

from sklearn.model_selection import ShuffleSplit    # bisa pilih beberapa teknik cross validation
from sklearn.model_selection import cross_val_score # untuk mengetahui performa model pada cross validation

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=50)

cv_accuracy = (cross_val_score(model, X_kbest_features, y, cv=cv, scoring='accuracy'))
avg_accuracy = np.mean(cv_accuracy)

print('Akurasi setiap split:', cv_accuracy, '\n')
print('Rata-rata akurasi pada cross validation:', avg_accuracy)

Akurasi setiap split: [0.78727842 0.76903024 0.79249218 0.78154327 0.78050052 0.7919708
 0.77632951 0.78258603 0.78727842 0.78467153] 

Rata-rata akurasi pada cross validation: 0.7833680917622523


In [ ]:
r = evaluation.shape[0]
evaluation.loc[r] = ['Logistic Regression', accuracy]
evaluation

,Model,accuracy
0,Naive Bayes,77.424400
1,Logistic Regression,79.405631


##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier     # Pilih salah satu algoritma supervised learning. Menggunakan algoritma Random Forest
from sklearn.model_selection import train_test_split    # Digunakan untuk memisahkan data uji dan data latih
from joblib import dump                                 # Digunakan untuk menyimpan model yang telah dilatih

In [ ]:
# Proses memisahkan data uji dan data latih. Perbandingan 80% untuk data latih, 20% untuk data uji
# Random_state digunakan untuk internal random generator
# Gunakan fitur (X) hasil seleksi fitur

# Split arrays or matrices into random train and test subsets.
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.2, random_state=40)

In [ ]:
# Training the model
algorithm = RandomForestClassifier()              # Load algoritma pembelajaran
model = algorithm.fit(X_train, y_train)   # Fitkan (latih) algoritma pada data latih & label latih 

# Simpan model hasil traning
dump(model, filename='model_3.joblib')

['model_3.joblib']

In [ ]:
# Gunakan model yang telah di latih untuk memprediksi label pada data uji
model_pred = model.predict(X_test)

# Tampilkan hasil prediksi label dari model
model_pred

array(['Netral', 'Netral', 'Positif', ..., 'Netral', 'Netral', 'Netral'],
      dtype=object)

In [ ]:
# Tampilkan label sebenarnya pada data uji (actual label)
y_test

array(['Netral', 'Netral', 'Positif', ..., 'Negatif', 'Netral', 'Negatif'],
      dtype=object)

In [ ]:
# Hitung jumlah data yang berhasil di prediksi model & jumlah data yang salah di prediksi
prediksi_benar = (model_pred == y_test).sum()
prediksi_salah = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

accuracy = prediksi_benar / (prediksi_benar + prediksi_salah)*100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 1556
Jumlah prediksi salah	: 362
Akurasi pengujian	: 81.12617309697602 %


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, model_pred)
print('Confusion matrix:\n', cm)

Confusion matrix:
 [[  45   83   23]
 [  25 1374   51]
 [  10  170  137]]


In [ ]:
from sklearn.metrics import classification_report

print('Classification report:\n', classification_report(y_test, model_pred))

Classification report:
               precision    recall  f1-score   support

     Negatif       0.56      0.30      0.39       151
      Netral       0.84      0.95      0.89      1450
     Positif       0.65      0.43      0.52       317

    accuracy                           0.81      1918
   macro avg       0.69      0.56      0.60      1918
weighted avg       0.79      0.81      0.79      1918



In [ ]:
# Cross Validation

from sklearn.model_selection import ShuffleSplit    # bisa pilih beberapa teknik cross validation
from sklearn.model_selection import cross_val_score # untuk mengetahui performa model pada cross validation

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=50)

cv_accuracy = (cross_val_score(model, X_kbest_features, y, cv=cv, scoring='accuracy'))
avg_accuracy = np.mean(cv_accuracy)

print('Akurasi setiap split:', cv_accuracy, '\n')
print('Rata-rata akurasi pada cross validation:', avg_accuracy)

Akurasi setiap split: [0.79666319 0.79144943 0.80291971 0.80917623 0.79457769 0.80865485
 0.80344108 0.79457769 0.79509906 0.80500521] 

Rata-rata akurasi pada cross validation: 0.8001564129301355


In [ ]:
r = evaluation.shape[0]
evaluation.loc[r] = ['Random Forest', accuracy]
evaluation

,Model,accuracy
0,Naive Bayes,77.424400
1,Logistic Regression,79.405631
2,Random Forest,81.126173
